In [1]:
!pip install opencv-python

In [2]:

class JudgeParticle:

    @staticmethod
    def get_particle_rate(df_row):
        img = df_row.waferMap
        particleCnt = np.count_nonzero(img == 2)
        particle_rate = particleCnt / df_row.dieSize
        return particle_rate

    @staticmethod
    def opening(image, non_zero_thread=4):
        cp = image.copy()
        for y in range(len(image)):
            for x in range(len(image[0])):
                if image[y][x] == 0:
                    continue
                offsets = [ (-1, 0),
                   (0, -1),           (0, 1),
                    (1, 0),
                ]
                values = []
                for offset in offsets:
                    _y = y + offset[0]
                    _x = x + offset[1]
                    if 0 <= _y < len(image) and 0 <= _x < len(image[0]):
                        values.append(image[_y][_x])
                    else:
                        values.append(0)
                if np.count_nonzero(values) <= non_zero_thread:
                    cp[y][x] = 0
                # cp[y][x] = np.count_nonzero(values)
        return cp
        
    @staticmethod
    def closing(image, non_zero_thread=1):
        cp = image.copy()
        for y in range(len(image)):
            for x in range(len(image[0])):
                if image[y][x] != 0:
                    continue
                offsets = [ (-1, 0),
                   (0, -1),           (0, 1),
                    (1, 0),
                ]
                values = []
                for offset in offsets:
                    _y = y + offset[0]
                    _x = x + offset[1]
                    if 0 <= _y < len(image) and 0 <= _x < len(image[0]):
                        values.append(image[_y][_x])
                    else:
                        values.append(0)
                if np.count_nonzero(values) >= non_zero_thread:
                    cp[y][x] = 2
                # cp[y][x] = np.count_nonzero(values)
        return cp

    @staticmethod
    def get_is_Scratch_accuracy(df_row):
        from myLabeling import myLabeling
        import math
        import cv2

        cutting_img = df_row.waferMap.copy()
        cutting_img[cutting_img < 2] = 0
        cutting_particle_cnt = np.count_nonzero(cutting_img)
        erosion_img = cutting_img
        erosion_img = JudgeParticle.opening(erosion_img, 2)
        erosion_particle_cnt = np.count_nonzero(erosion_img)
        rate = erosion_particle_cnt / cutting_particle_cnt
        return rate
        
    @staticmethod
    def is_Scratch(df_row, is_print=True, thread=0.017):
        from myLabeling import myLabeling
        import math
        import cv2

        rate = JudgeParticle.get_is_Scratch_accuracy(df_row)
        if rate <= thread:
            return True
        else:
            return False
        
    @staticmethod
    def is_Center(df_row, is_print=True):
        from myLabeling import myLabeling
        import math
        import cv2

        labelObj = myLabeling.preprocess_detail(df_row.waferMap)
        x, y, w, h, size = labelObj['data']
        cutting_img = labelObj['labelMap']
        cutting_img[cutting_img < 2] = 0
        M = cv2.moments(cutting_img.astype('uint8'), False) # 重心を求める　前処理
        x,y = int(M["m10"]/M["m00"]) , int(M["m01"]/M["m00"]) # 重心を求める
        c_x, c_y = len(cutting_img[0])/2, len(cutting_img)/2
        distance = math.sqrt((x - c_x)**2 + (y - c_y)**2)
        avg_distance = (c_x + c_y) / 2 / 4
        # if is_print:
        #     print(distance, avg_distance, x, y, c_x, c_y)
        if distance < avg_distance:
            return True
        return False
        
    @staticmethod
    def is_Random(df_row):
        particle_rate = JudgeParticle.get_particle_rate(df_row)
        if 0.38 < particle_rate < 0.72:
            return True
        return False

    @staticmethod
    def is_Near_full(df_row):
        particle_rate = JudgeParticle.get_particle_rate(df_row)
        if particle_rate >= 0.72:
            return True
        return False


In [3]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from skimage.transform import resize
import matplotlib.pyplot as plt
from PIL import Image
from ipywidgets import interact, Select
from myLabeling import myLabeling
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# 最も良いScratchのthreadを求める。１％以下の誤検出を良いものとする。

# filterd_df = df[df.failureType == 'Loc']
# filterd_df_accuracies = []
# for row_df in filterd_df.itertuples():
#     rate = JudgeParticle.get_is_Scratch_accuracy(row_df)
#     filterd_df_accuracies.append(rate)

# print('start')

# for run_cnt in range(1, 1000):
#     scratch_rate = []
#     thread = run_cnt * 0.001
#     for accuracy in filterd_df_accuracies:
#         if accuracy <= thread:
#             scratch_rate.append(2)
#         else:
#             scratch_rate.append(0)
#     total_cnt = len(scratch_rate)
#     true_cnt = np.count_nonzero(scratch_rate)
#     print('\r閾値: ', thread, end="")
#     if true_cnt / total_cnt > 0.01:
#         break

# print('\n最終', thread)


In [4]:

import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from skimage.transform import resize
import matplotlib.pyplot as plt
from PIL import Image
from ipywidgets import interact, Select
from myLabeling import myLabeling
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

df=pd.read_pickle("../input/LSWMD_25519.pkl")

def image_show(target_df, x=0):
    ori_image = target_df.waferMap[x]
    trans_image = ori_image
    trans_image = myLabeling.preprocess(trans_image)
    plt.imshow(trans_image)
    plt.show()
    plt.imshow(ori_image)
    plt.show()
    return x

def select_type(failureType='Loc'):
    filterd_df = df[df.failureType == failureType]
    # scratch_rate = []
    # for row_df in filterd_df.itertuples():
    #     if JudgeParticle.is_Scratch(row_df):
    #         scratch_rate.append(2)
    #     else:
    #         scratch_rate.append(0)
    # print(len(filterd_df), len(scratch_rate))
    # plt.hist(scratch_rate)
    # filterd_df = filterd_df.drop(index=drop_indexes)
    num_widget = widgets.IntSlider(value=0, min=0, max=len(filterd_df.waferMap) - 1)
    interact(image_show, x=num_widget, target_df=fixed(filterd_df.reset_index()));
    # return failureType

#スライダー
labels = df.failureType.unique()
interact(select_type, failureType=labels);

interactive(children=(Dropdown(description='failureType', options=('Loc', 'Edge-Loc', 'Center', 'Edge-Ring', '…